<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Mohammad Abolnejadian
### Student Number : 98103867
### Colab Link: https://colab.research.google.com/drive/1CDZPV6574prYINZK0xYFFp4JxGVRcBK4?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [105]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 19:41:21--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 19:41:21--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc455f335acbc180dd7a88857df4.dl.dropboxusercontent.com/cd/0/get/B-xg_DryKUzb4lGOnKO9ynaq3kbyKYVY9yMQR7k9-mwyd6Ri5eHL_cx2gSGddYvmSW8JwgDbC6cRlqOi-0Zc-f_YeL9uH9ffPDjMxPVx6pO9LNSvGD3PknxaeJbIxkBuQfmsc7RYinK3Mg1jeQqlXa9qfIT1WPp77Vjd9iNwSg16tkMwQufChXVKTXHXfpCOKjY/file?dl=1# [following]
--2023-06-27 19:41:22--  https://uc455f335acbc180dd7a88857df4.dl.dropboxusercontent.com/cd/0/get/B-xg_DryKUzb4lGOnKO9ynaq3kbyKYVY9yMQR7k9-mwyd6Ri5eHL_cx2gSGd

In [106]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [107]:
import zipfile #Was neccessary to extract the zip file that was downloaded
import re #Was neccessary to sort file paths
x = []
def sort_file_paths_by_number(file_paths):
  """Sorts an array of file paths based on the number in the file path.

  Args:
    file_paths: An array of file paths.

  Returns:
    A sorted array of file paths.
  """

  def extract_number(file_path):
    """Extracts the number from the file path."""
    match = re.search(r"\d+", file_path)
    if match:
      return int(match.group(0))
    else:
      return None

  sorted_file_paths = sorted(file_paths, key=extract_number)
  return sorted_file_paths

image_extensions = ('.jpg', '.jpeg', '.png')
with zipfile.ZipFile('images.zip', "r") as zip_data:
  # print(zip_data.namelist())
  file_names = list(filter(lambda x:x.endswith(image_extensions), zip_data.namelist()))
  file_names = sort_file_paths_by_number(file_names)
  print(file_names)
  for file_name in file_names:
    image = Image.open(zip_data.open(file_name))
    image_array = np.asarray(image)
    x.append(image_array)
  x = np.array(x)
meta_df = pd.read_csv('meta.csv')
print(x)
meta_df

['content/img-data/0.jpg', 'content/img-data/1.jpg', 'content/img-data/2.jpg', 'content/img-data/3.jpg', 'content/img-data/4.jpg', 'content/img-data/5.jpg', 'content/img-data/6.jpg', 'content/img-data/7.jpg', 'content/img-data/8.jpg', 'content/img-data/9.jpg', 'content/img-data/10.jpg', 'content/img-data/11.jpg', 'content/img-data/12.jpg', 'content/img-data/13.jpg', 'content/img-data/14.jpg', 'content/img-data/15.jpg', 'content/img-data/16.jpg', 'content/img-data/17.jpg', 'content/img-data/18.jpg', 'content/img-data/19.jpg', 'content/img-data/20.jpg', 'content/img-data/21.jpg', 'content/img-data/22.jpg', 'content/img-data/23.jpg', 'content/img-data/24.jpg', 'content/img-data/25.jpg', 'content/img-data/26.jpg', 'content/img-data/27.jpg', 'content/img-data/28.jpg', 'content/img-data/29.jpg', 'content/img-data/30.jpg', 'content/img-data/31.jpg', 'content/img-data/32.jpg', 'content/img-data/33.jpg', 'content/img-data/34.jpg', 'content/img-data/35.jpg', 'content/img-data/36.jpg', 'content/i

,path,label
0,0.jpg,jones
1,1.jpg,jones
2,2.jpg,jones
3,3.jpg,jones
4,4.jpg,jones
...,...,...
395,395.jpg,vaux
396,396.jpg,vaux
397,397.jpg,vaux
398,398.jpg,vaux


Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [108]:
x_flatten = []
for image in x:
  x_flatten.append((image/255).flatten())
x = np.array(x_flatten).copy()
x.shape

(400, 4096)

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [109]:
le = LabelEncoder()
encoded_column = le.fit_transform(meta_df['label'].values)
y = np.array(encoded_column)
y

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 33, 33, 33, 33, 33, 33, 33,
       33, 33, 33,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 24,
       24, 24, 24, 24, 24

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [110]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1, stratify = y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=(1/9), random_state=1, stratify = y_train) #1/9 * 0.9 = 0.1
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((320, 4096), (40, 4096), (40, 4096), (320,), (40,), (40,))

# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [111]:
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid
## PARAMETER SETTING
C_parameter = np.logspace(-1, 3, 5)
kernel_parameter = ['linear', 'rbf', 'poly', 'sigmoid']
gamma_parameter = np.logspace(-4, 1, 5)
tolerance_parameter = np.logspace(-6, -3, 4)
tuning_parameters = {'C':C_parameter,
                     'kernel': kernel_parameter,
                     'gamma': gamma_parameter,
                     'tolerance': tolerance_parameter}
# degree_parameter = np.arange(1, 6) #only applicable in poly kernel
# coef_parameter = np.logspace(-1, 1, 3) #only applicable in poly and sigmoid kernels

## PARAMETER TUNING
best_score = 0
best_params = dict()
param_grid = ParameterGrid(tuning_parameters)
for param in param_grid:
  sklearn_svm_tuning = SVC(C = param['C'], kernel = param['kernel'], gamma = param['gamma'], tol = param['tolerance'])
  sklearn_svm_tuning.fit(x_train, y_train)
  score = sklearn_svm_tuning.score(x_val, y_val)
  if score > best_score:
    best_score = score
    best_params = param

## LEARNING
sklearn_svm = SVC(C = best_params['C'], kernel = best_params['kernel'], gamma = best_params['gamma'], tol = best_params['tolerance'])
sklearn_svm.fit(x_train, y_train)
sklearn_svm.get_params()

{'C': 0.1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.0001,
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 1e-06,
 'verbose': False}

Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [112]:
from sklearn.metrics import f1_score
def f1_single_class(actual, predicted, label):
    """ A helper function to calculate f1-score for the given `label` """

    tp = np.sum((actual==label) & (predicted==label))
    fp = np.sum((actual!=label) & (predicted==label))
    fn = np.sum((predicted!=label) & (actual==label))

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)

    if precision != 0 and recall != 0:
      f1 = (2 * precision * recall) / (precision + recall)
    else:
      f1 = 0
    # f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def calculate_f1(actual, predicted):
    return np.mean([f1_single_class(actual, predicted, label)
        for label in np.unique(actual)])

sklearn_svm_predictions = sklearn_svm.predict(x_test)
# print(sklearn_svm_predictions)
print(f'accuracy: {sklearn_svm.score(x_test, y_test)}')
f1_score = f1_score(y_test, np.array(sklearn_svm_predictions), average = 'macro')
# f1_score_manual = calculate_f1(y_test, np.array(sklearn_svm_predictions))
print(f'f1: {f1_score}')

accuracy: 1.0
f1: 1.0


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [113]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False
class SVM:
  def linear_kernel(self, x1, x2):
    return np.inner(x1, x2)

  def poly_kernel(self, x1, x2):
    return (np.inner(x1, x2) + self.coef0)**self.degree

  def rbf_kernel(self, x1, x2):
    distance = np.linalg.norm(x1 - x2) ** 2
    return np.exp(-self.gamma * distance)

  def get_kernel_matrix(self, X):
    kernel_matrix = np.zeros((self.n_samples, self.n_samples))
    for i in range(self.n_samples):
      for j in range(self.n_samples):
        kernel_matrix[i, j] = self.kernel_method(X[i], X[j])
    return kernel_matrix

  def get_QP_matrices(self, X, y, kernel_matrix):
    P = matrix(np.outer(y, y) * kernel_matrix, tc='d')
    q = matrix(np.ones(self.n_samples) * -1)
    # A = matrix(y, (1, self.n_samples), tc='d')
    A = matrix(y.reshape(1, -1), tc = 'd')
    b = matrix(0, tc='d')
    if self.C:
      G = matrix(np.vstack((np.identity(self.n_samples) * -1, np.identity(self.n_samples))))
      # h = matrix(np.vstack((np.zeros(self.n_samples), np.ones(self.n_samples) * self.C)))
      h = matrix(np.hstack((np.zeros(self.n_samples), np.ones(self.n_samples) * self.C)))
    else:
      G = matrix(np.identity(self.n_samples) * -1)
      h = matrix(np.zeros(self.n_samples))
    return P, q, A, b, G, h

  def __init__(self, C=1, kernel='linear', degree=3, gamma=None, coef0=0):
    #defaulted to sklearn defaults
    self.C = C
    self.kernel = kernel
    self.degree = degree
    self.gamma = gamma
    self.coef0 = coef0
    self.n_samples = None
    self.n_features = None
    self.alphas = None
    self.support_vectors = None
    self.support_vectors_y = None
    self.intercept = None

  def fit(self, X, y):
      """Fit the SVM model to dataset X in one-vs-one manner.

      Parameters
      ----------
      X : array-like of shape (n_samples, n_features)
          Training data, where `n_samples` is the number of samples
          and `n_features` is the number of features.

      y : array-like of shape (n_samples,)
          Training labels, where `n_samples` is the number of samples.

      Returns
      -------
      self : object
          Returns the instance itself.
      """

      # STEPS (For each of the classifiers you would need to train)
      # 1. Setup SVM objective function in QP form
      # 2. Solve the QP problem and get the optimal lagrange multipliers
      # 3. Get support vectors and calculate the weight vector
      self.n_samples, self.n_features = np.shape(X)

      if not self.gamma:
        #default value set in sklearn package
        self.gamma = 1 / self.n_features

      #Initialize kernel
      if self.kernel.lower() == 'rbf':
        self.kernel_method = self.rbf_kernel
      elif self.kernel.lower() == 'poly' or self.kernel.lower() == 'polynomial':
        self.kernel_method = self.poly_kernel
      else:
        self.kernel_method = self.linear_kernel

      kernel_matrix = self.get_kernel_matrix(X)

      #Optimization based on the following formula:
      #minimize (1/2)x^TPx+q^Tx s.t Ax=b & Gx<=h
      P, q, A, b, G, h = self.get_QP_matrices(X, y, kernel_matrix)
      solution = solvers.qp(P, q, G, h, A, b)
      alphas = np.ravel(solution['x'])
      idx = alphas > 1e-4
      self.support_vectors = X[idx]
      self.support_vectors_y = y[idx]
      self.alphas = alphas[idx]

      #Calculate bias (can't have the weight matrix since we don't have the transformed data yet)
      self.intercept = self.support_vectors_y[0]
      for i in range(len(self.alphas)):
        self.intercept -= self.alphas[i] * self.support_vectors_y[i] * self.kernel_method(self.support_vectors[i], self.support_vectors[0])

      return self

  def predict(self, x):
    prediction = 0
    # Determine the label of the sample by the support vectors
    for i in range(len(self.alphas)):
      prediction += self.alphas[i] * self.support_vectors_y[i] * self.kernel_method(self.support_vectors[i], x)
    prediction += self.intercept
    return np.sign(prediction)


  def transform(self, X):
      """Fit the model with X and apply the dimensionality reduction on X.

      Parameters
      ----------
      X : array-like of shape (n_samples, n_features)
          Training data, where `n_samples` is the number of samples
          and `n_features` is the number of features.

      Returns
      -------
      y_pred : ndarray of shape (n_samples)
          Your predictions for the class of each sample
      """
      y_pred = [self.predict(x) for x in X]
      return np.array(y_pred)

Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [114]:
import itertools
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score
# def convert_raw_model_prediction(model_prediction, model):
#   return

#Get all possible pairwise possiblities
one_vs_one_pairs = list(itertools.combinations(np.unique(y_train), 2))

models = []
#Train all pairwise models
for one_vs_one_pair in one_vs_one_pairs:
  pair_idx = np.append(np.where(y_train == one_vs_one_pair[0])[0], np.where(y_train == one_vs_one_pair[1])[0])
  pair_x = x_train[pair_idx]
  pair_y = y_train[pair_idx]
  transform_func = np.vectorize(lambda cl: -1 if cl==one_vs_one_pair[0] else 1)
  pair_y = transform_func(pair_y)

  models.append((SVM().fit(pair_x, pair_y), one_vs_one_pair[0], one_vs_one_pair[1]))

predictions = []
for model in models:
  model_predictions = model[0].transform(x_test)
  transform_func = np.vectorize(lambda cl: model[1] if cl == -1 else model[2])
  model_predictions = transform_func(model_predictions)
  predictions.append(model_predictions.tolist())

predictions = np.array(predictions)
y_pred = []
for i in range(len(x_test)):
  y_pred.append(Counter(predictions[:, i]).most_common(1)[0][0])
print(y_pred)
print(f"accuracy score: {accuracy_score(y_test, np.array(y_pred))}")
print(f"f1 score: {f1_score(y_test, np.array(y_pred), average = 'macro')}")

[16, 28, 13, 18, 30, 3, 15, 14, 34, 25, 2, 36, 12, 17, 8, 38, 37, 30, 31, 27, 33, 24, 35, 29, 7, 11, 5, 6, 23, 1, 22, 32, 10, 21, 26, 0, 9, 4, 20, 19]
accuracy score: 1.0
f1 score: 1.0


# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [115]:
class PCA:
    def __init__(self, n_components = 3):
        self.n_components = n_components
        self.components_ = None
        self.mean = None

    def fit(self, X):
        """Fit the model with X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        self.components_ = None
        self.mean = np.mean(X, axis=0)
        #mean centering the data
        X = X - self.mean

        cov = np.cov(X.T)

        #eigenvalues and eigenvectors of the covariance matrix
        eigenvalues, eigenvectors = np.linalg.eig(cov)

        idxs = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idxs]
        eigenvectors = eigenvectors[:, idxs]

        self.components_ = eigenvectors[:, :self.n_components]


        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_components)
            Transformed values.
        """
        assert self.components_ is not None

        X = X - self.mean
        return np.dot(X, self.components_)

In [116]:
names = ['jones', 'taylor', 'anderson', 'wilson']
transformed_xs = []
transformed_ys = []
for name in names:
  transformed = le.transform([name])[0]
  idx = np.where(y == transformed)
  transformed_xs.extend(x[idx])
  transformed_ys.extend(y[idx])

transformed_ys = np.array(transformed_ys)
transformed_xs = np.array(transformed_xs)
pca = PCA(3)
pca = pca.fit(transformed_xs)
transformed_xs = pca.transform(transformed_xs)
transformed_xs = np.real(transformed_xs)
transformed_xs.shape

(40, 3)

Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [117]:
transformed_named_ys = le.inverse_transform(transformed_ys)
# transformed_named_ys.shape
transformed_df_data = np.concatenate((transformed_xs, transformed_named_ys.reshape(-1, 1)), axis = 1)
# transformed_df_data
transformed_df = pd.DataFrame(transformed_df_data, columns = ['component 1', 'component 2', 'component 3', 'names'])
fig = px.scatter_3d(transformed_df, x='component 1', y='component 2', z='component 3',
              color='names')
fig.show()

# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [118]:
class KMeans:
    """
    Fits it on data, then uses predict to get cluster labels.
    """

    def __init__(self, k, max_iters = 500):
        self.k = k
        self.max_iters = max_iters
        self.centroids = None

    def closest_centroid_idx(self, x):
      distances = np.linalg.norm(x - self.centroids, axis=1)
      return np.argmin(distances)

    def cluster(self, X, n_samples):
      clusters = []
      for i in range(n_samples):
        clusters.append(self.closest_centroid_idx(X[i]))
      return np.array(clusters)

    def get_new_centroids(self, X, clusters):
      centroids = []
      for i in range(self.k):
        cluster_samples = X[clusters == i]
        cluster_centroid = np.mean(cluster_samples, axis=0)
        centroids.append(cluster_centroid)
      return np.array(centroids)

    def fit(self, X, y):
        """Fits the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        centroids_idx = np.random.choice(n_samples, self.k, replace=False)
        self.centroids = X[centroids_idx, :]

        for _ in range(self.max_iters):
          clusters = self.cluster(X, n_samples)

          previous_centroids = self.centroids

          self.centroids = self.get_new_centroids(X, clusters)

          diff = self.centroids - previous_centroids
          if not diff.any(): #converged
            break


        return self

    def predict(self, X):
        """ Predicts the cluster label
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted cluster label
        """

        y_pred = self.cluster(X, X.shape[0])
        return y_pred

In [119]:
kmeans = KMeans(k=4)
kmeans.fit(transformed_xs, transformed_ys)
kmeans_predictions = kmeans.predict(transformed_xs)
kmeans_predictions

array([3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 1, 1, 3, 3, 0, 2, 0, 3, 3, 1, 3, 0, 2, 3])

In [120]:
transformed_df = transformed_df.drop('names', axis = 1)
transformed_df['clusters'] = kmeans_predictions
fig = px.scatter_3d(transformed_df, x='component 1', y='component 2', z='component 3',
              color='clusters')
fig.show()

Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [121]:
from sklearn.metrics import silhouette_score
silhouette_score(transformed_xs, kmeans_predictions, metric='euclidean')

0.37043213823018223

Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

**As it can be seen in this particular example, the answer to the first question is negative. K-means clusters the dataset in an unsupervised way and it can be seen that it overfits to the features and the difference between features of each datapoint.**
**On the hand, true labels are different and it is more spread amongst the diagram. But sometimes the answer to the first question can be positive, since the original dataset is much more clustered center some centroids.**
**The conclusion is that in K-means, which is unsupervised, ignoring labels can be dangerous since the only knowledge we can get from data is their feature and the distance between them. Using supervised learning methods will generally give out a better result (in terms of bias and accuracy)**

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [122]:
x_nn = x.reshape(-1, 64, 64, 1)
x_nn.shape

(400, 64, 64, 1)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [123]:
shift_range = 0.2
data_augmentation = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=shift_range,
    height_shift_range=shift_range,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4,1.5],
    zoom_range=0.3,
    fill_mode='nearest'
)

# data_augmentation = keras.Sequential([
#     layers.experimental.preprocessing.RandomFlip(),
#     layers.experimental.preprocessing.RandomRotation(0.2),
#     layers.experimental.preprocessing.RandomContrast(0.5),
#     layers.experimental.preprocessing.RandomWidth(0.4),
#     layers.experimental.preprocessing.RandomHeight(0.2),
# ])
data_augmentation_generator = data_augmentation.flow(x_nn, y, batch_size=128)
augmented_x_nn = next(data_augmentation_generator)[0].astype('uint8')
augmented_y = next(data_augmentation_generator)[1].astype('uint8')
augmented_x_nn.shape, augmented_y.shape

((128, 64, 64, 1), (128,))

In [124]:
images = np.concatenate([x_nn, augmented_x_nn])
labels = np.concatenate([y, augmented_y])
images.shape, labels.shape

((528, 64, 64, 1), (528,))

In [125]:
len(np.unique(labels))

39

In [126]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=1)

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [127]:
nn_model = keras.models.Sequential([
    keras.layers.Conv2D(32, (1, 1), activation='relu', input_shape=(64, 64, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(39, activation='softmax') #since we have 39 unique labels
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(train_images, train_labels, epochs=100, batch_size=128)

Epoch 1/100
4/4 [==============================] - 5s 598ms/step - loss: 3.6730 - accuracy: 0.0284
Epoch 2/100
4/4 [==============================] - 3s 839ms/step - loss: 3.6634 - accuracy: 0.0190
Epoch 3/100
4/4 [==============================] - 6s 1s/step - loss: 3.6675 - accuracy: 0.0379
Epoch 4/100
4/4 [==============================] - 4s 714ms/step - loss: 3.6630 - accuracy: 0.0427
Epoch 5/100
4/4 [==============================] - 3s 592ms/step - loss: 3.6614 - accuracy: 0.0237
Epoch 6/100
4/4 [==============================] - 2s 579ms/step - loss: 3.6609 - accuracy: 0.0332
Epoch 7/100
4/4 [==============================] - 3s 581ms/step - loss: 3.6611 - accuracy: 0.0403
Epoch 8/100
4/4 [==============================] - 4s 977ms/step - loss: 3.6583 - accuracy: 0.0474
Epoch 9/100
4/4 [==============================] - 3s 580ms/step - loss: 3.6609 - accuracy: 0.0403
Epoch 10/100
4/4 [==============================] - 2s 576ms/step - loss: 3.6595 - accuracy: 0.0450
Epoch 11/100

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [128]:
nn_model.evaluate(test_images, test_labels)

4/4 [==============================] - 0s 37ms/step - loss: 1.3102 - accuracy: 0.6981


[1.3102439641952515, 0.698113203048706]